## CMSEtech Webscraper

### Extract CMSE classes information from a local file Using Beautiful Soup

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import re

In [ ]:
with open('./SS23_CMSE201.html') as f:
    html_doc = f.read()

soup = BeautifulSoup(html_doc, 'html.parser')
"""tables = soup.findAll("table", {"class": "ps_grid-flex"})
tab = tables[0]
trs = tab.findAll('tr')"""

In [ ]:
values = []
divs = soup.find_all("div", class_="ps-htmlarea")
for div in divs:
    val = (div.get_text(strip=True))
    values.append(val)
values = list(filter(lambda x: x != "", values))
values.pop(0)
values

In [ ]:
def get_class_info(values):
    reshaped_list = [values[i:i+6] for i in range(0, len(values), 6)]

    # Create a DataFrame from the reshaped list
    col_names = ['Course', 'Type', 'Section', 'Schedule', 'Dates', 'Instructor']
    df = pd.DataFrame(reshaped_list, columns=col_names)
    df[['Course Code', 'Course Name']] = df['Course'].str.split(':', 1, expand=True)
    df[['Type', 'Units']] = df['Type'].str.split('(', 1, expand=True)
    df[['Section', 'Class Nbr', 'Academic Session']] = df['Section'].str.split('/', 2, expand=True)
    df[['Days', 'Time']] = df['Schedule'].str.split(':', 1, expand=True)
    df[['Units','Status']] = df['Units'].str.split(')',1,expand=True)
    df[['Subject','Course Number']] = df['Course Code'].str.split(' ',1,expand=True)

    df = df.drop(['Course', 'Schedule','Course Code'], axis=1)
    df = df[['Subject','Course Number','Course Name','Type','Units','Status','Section','Class Nbr','Academic Session','Days','Time','Dates','Instructor']]
    df['Units'] = df['Units'].str.extract(r'(\d+(?:\.\d+)?)')
    df['Section'] = df['Section'].str.extract(r'(\d+(?:\.\d+)?)')
    df['Class Nbr'] = df['Class Nbr'].str.extract(r'(\d+(?:\.\d+)?)')
    return df
df = get_class_info(values)
df